In [85]:
#------------------#
# Import libraries #
#------------------#

import codecs
import codecs
import csv
import pandas as pd
import numpy as np
import json     
import os
import fnmatch
import os

In [86]:
#-------------------------#
# Set directory structure #
#-------------------------#

homedir = '/Users/emilyhahn/projects/tlc/'
directory = os.path.join(homedir, 'behavior_raw_4-15-18')
output = os.path.join(homedir,'behavior_preprocessed_4-15-18')

In [87]:
#----------------------------#
# Generate subject list: MGH #
#----------------------------#

# subjlist=[]

# for file in os.listdir(directory):
#     if fnmatch.fnmatch(file, '*.txt'):
#         if file.startswith('Learning_Trial-'):
#             file=file.rstrip('.txt')
#             file = file.split('-', 1)[1]
#             if file.startswith("02"):
#                 subjlist.append(file)  

#--------------------------------------#
# Generate subject list: Chicago, UCLA #
#--------------------------------------#

subjlist=[]

# for file in os.listdir(directory):

#     if file.startswith('03'): # Chicago
    
#     if file.startswith('01'): # UCLA
        
#         if file.endswith('_Behavior'):
#             file=file.rstrip('_Behavior')
#         elif file.endswith('_BEHAVIOR'):
#             file=file.rstrip('_BEHAVIOR')
#         else:
#             print('check directory name for %s' % file)
#         file=file[:-4]
#         subjlist.append(file)

#---------------------------------------#
# Read subject list: MGH, Chicago, UCLA #
#---------------------------------------#

csv=pd.read_csv(os.path.join(directory, 'subjlist.csv'))
for i in csv:
    subjlist.append(i)    

In [88]:
def edat2csv(subjid, directory, output):
    
    #-------------------------------#
    # Pull info from edat .txt file #
    #-------------------------------#
        
    # MGH
#     with codecs.open(os.path.join(directory,'Learning_Trial-%s.txt' % subjid), encoding='utf-16') as f:
    
    # Chicago
#     chicago = os.path.join(directory, '%sMR01_Behavior'% subjid,'Learning_Trial-%s-1.txt' % subjid)
    
    # UCLA
    ucla = os.path.join(directory, '%sMR01_BEHAVIOR'% subjid,'Learning_Trial-%s-1.txt' % subjid)
#     ucla = os.path.join(directory, '%sMR01_Behavior'% subjid,'Learning_Trial-%s-01.txt' % subjid)
   
    with codecs.open(ucla, encoding='utf-16') as f:  
        
        cue_col=[]
        RT_col=[]
        cresp_col=[]
        resp_col=[]

        for line in f:
            if ':' in line:
                value = line.split()     
                if 'Cue:' in line:
                    cue = value
                    cue_col.append(cue)
                elif 'Cue.RT:' in line:
                    RT = value
                    RT_col.append(RT)
                elif 'Cue.CRESP:' in line:
                    cresp = value
                    cresp_col.append(cresp)
                elif 'Cue.RESP:' in line:
                    resp = value
                    resp_col.append(resp)            
        
        cue_1=[]
        RT_1=[]
        cresp_1=[]
        resp_1=[]  

        def append_cols(orig,new):
            for i in orig:
                new.append(i[1])
            return

        append_cols(cue_col, cue_1)
        append_cols(RT_col, RT_1)

        
        def add_zeroes(col1, col2):
            for i in col1:
                del(i[0])

            for i in col1:
                if i:
                    col2.append(i)
                if not i:
                    col2.append('0')
            return
        
        add_zeroes(resp_col, resp_1)
        add_zeroes(cresp_col, cresp_1)
        
    #-------------------#
    # Strip JSON format #
    #-------------------#
                
        CRESP_1=[]
        RESP_1=[]
        
        def strip_json(encoded, stripped):
        
            for i in encoded:
                i=json.dumps(i)
                i=i.strip("[]")
                stripped.append(i)
            
            return
        
        strip_json(resp_1, RESP_1)
        strip_json(cresp_1, CRESP_1)
        

    #-------------------------------------#
    # Write analysis compatible .txt file #
    #-------------------------------------#
        
        def gen_df(subjid, c_1, c_2, c_3, c_4):
            
            df=pd.DataFrame()
            
            subjid = subjid.split('-',1)[0]
            
            sub = np.array([subjid]*533)
            
            df['subjid']=sub
            df['Image']=c_1
            df['Correct']=c_2
            df['ResponseType']=c_3
            df['ResponseTime']=c_4
                        
            for i, col in enumerate(df.columns):
                df.iloc[:, i] = df.iloc[:, i].str.replace('"', '')
            
            x='%s/%s.csv' % (output, subjid)
                        
            out = pd.DataFrame.to_csv(df, x, index=None)
                              
            print('Done: %s' % subjid)
            
            return
        
        gen_df(subjid, cue_1, CRESP_1, RESP_1, RT_1)


    return

In [89]:
#--------------------------------#
# Convert edat .txt file to .csv #
#--------------------------------#

for subjid in subjlist:
    edat2csv(subjid, directory, output)

Done: 010117
